In [2]:
import warnings
warnings.filterwarnings('ignore')


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import MySQLdb
from sqlalchemy import create_engine

import tools.feature_selection as fs
import tools.preprocessing as ps
import tools.evaluations as ev

## Data Import

### Import data from Database

In [2]:
host = 'db1.ceobypqfxmry.us-east-2.rds.amazonaws.com'
port = '3306'
db = 'db'
user = 'admin'
password = 'msbd5001'
tb = 'ip_attr2'
engine = create_engine(str(r"mysql+mysqldb://%s:" + '%s' + "@%s/%s") % (user, password, host, db))

In [7]:
df_train = pd.read_sql_table(tb, engine)
df_train.head()

OperationalError: (MySQLdb._exceptions.OperationalError) (2026, 'SSL connection error: SSL_CTX_set_tmp_dh failed')
(Background on this error at: http://sqlalche.me/e/e3q8)

In [8]:
# save data as csv file
df_train.to_csv('./data/ip_attr2.csv', index=False)

NameError: name 'df_train' is not defined

### Import data from csv file

In [3]:
#df_train = pd.read_csv('./data/pa_attr.csv')
df_train = pd.read_csv(r'/Users/yin/Desktop/5001/project/home-credit-default-risk/db_ip_attr2.csv')

## Data Profiling

In [ ]:
import pandas_profiling
profile = pandas_profiling.ProfileReport(df_train, check_correlation=False)
profile.to_file('./data_profile/ip_attr2.html')

## Data Processing

### Separating continuous and categorical variables

In [4]:
# Separating continuous and categorical variables (setting yout own rule)
cate_vars = []
cont_vars = []
for col in df_train.columns:
    if col not in ['SK_ID_CURR1','TARGET']:
        if df_train[col].dtype in ['float64','int64'] and df_train[col].nunique() > 3:
            cont_vars.append(col)
        else:
            cate_vars.append(col)

In [ ]:
# Encoding categorical variables
df_train = ps.cate_enc(df_train, cate_vars)

### Missing Values

In [5]:
# fill in missing values
# if all values of the column are positive -> -1
# if all values of the columns are negative -> 1
all_pos = []
all_neg = []
both = []
for var in cont_vars:
    if df_train[var].max() <= 0:
        df_train[var].fillna(0xdeadbeef, inplace=True)
        all_neg.append((var,(df_train[var].min(), df_train[var].max())))
    elif df_train[var].min() >= 0:
        df_train[var].fillna(0xdeadbeef, inplace=True)
        all_pos.append((var,(df_train[var].min(), df_train[var].max())))
    else:
        both.append((var,(df_train[var].min(), df_train[var].max())))

## Feature Selection

In [6]:
#df_train_woe = df_train.fillna(-5)
df_train_woe = df_train.copy()
df_train_woe['target'] = df_train_woe['TARGET']
df_train_woe.drop('TARGET',axis=1,inplace=True)

In [ ]:
df_train_woe.shape

In [7]:
df_iv = fs.cal_iv(df_train_woe,cate_vars,cont_vars,'target')

-----------process continuous variable:SK_ID_PREV-----------
process continuous variable:installments_payments_NUM_INSTALMENT_VERSION_min
process continuous variable:installments_payments_NUM_INSTALMENT_VERSION_max
process continuous variable:installments_payments_NUM_INSTALMENT_VERSION_avg
process continuous variable:installments_payments_NUM_INSTALMENT_VERSION_std
process continuous variable:installments_payments_NUM_INSTALMENT_NUMBER_min
process continuous variable:installments_payments_NUM_INSTALMENT_NUMBER_max
process continuous variable:installments_payments_NUM_INSTALMENT_NUMBER_avg
process continuous variable:installments_payments_NUM_INSTALMENT_NUMBER_std
process continuous variable:installments_payments_DAYS_INSTALMENT_min
process continuous variable:installments_payments_DAYS_INSTALMENT_max
process continuous variable:installments_payments_DAYS_INSTALMENT_avg
process continuous variable:installments_payments_DAYS_INSTALMENT_std
process continuous variable:installments_paymen

In [9]:
df_iv.to_excel('./features/df_ip_iv.xlsx',index=False)

In [ ]:
df_iv = pd.read_excel('./features/df_train_iv.xlsx')
iv_features_train = df_iv[df_iv['iv']>0.02]['var_name'].values

## Modeling

In [ ]:
# fill in missing values.
df_train.fillna(-1,inplace=True)

In [ ]:
# encoding TARGET
df_train['TARGET'] = df_train['TARGET'].astype('category')

In [ ]:
# split dataset
X_train, X_test, y_train, y_test = ps.split_data(df_train,iv_features_train,'TARGET')

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# model training
model = RandomForestClassifier(max_depth=9,n_estimators=100)
model.fit(X_train, y_train)

# output probability
y_hat = model.predict_proba(X_train)[:,1]
y_pred = model.predict_proba(X_test)[:,1]

# evaluation
ev.plot_ROC(y_train, y_hat, 'Train')
ev.plot_ROC(y_test, y_pred, 'Test')